# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [3]:
#Definition of the tables.
import pandas as pd
# Table and definitions sample
data = {
    'table': ['employees', 'salary', 'studies'],
    'definition': [
        'Employee information, name, salary, department, position',
        'Salary details for each year, salary, year, bonus, deductions',
        'Educational studies, name of institution, GPA score, degree, year of graduation'
    ]
}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0  employees  Employee information, name, salary, department...
1     salary  Salary details for each year, salary, year, bo...
2    studies  Educational studies, name of institution, GPA ...


In [4]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [5]:
print(text_tables)

employees: Employee information, name, salary, department, position
salary: Salary details for each year, salary, year, bonus, deductions
studies: Educational studies, name of institution, GPA score, degree, year of graduation


In [6]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [7]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="What is the average salary of all employees?")

In [8]:
print(return_OAI(pqt1))

{
    "tables": ["employees", "salary"]
}


In [9]:
# Example 2: Education-related query
pqt2 = prompt_question_tables.format(
    tables=text_tables,
    question="Which employees have a GPA above 3.5?"
)
print(return_OAI(pqt2))


{
    "tables": ["employees", "studies"]
}


In [10]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="What is the average salary of employees who have a master's degree?")

In [11]:
print(return_OAI(pqt3))

{
    "tables": ["employees", "salary"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [13]:
#Creating the prompt, with the user questions and the tables definitions.
# Example 3: Complex query involving all tables
pqt4 = prompt_question_tables.format(
    tables=text_tables,
    question="What is the average bonus for employees with a Computer Science degree who make above $100,000?"
)
print("Test Case 3 - Complex Query:")
print(return_OAI(pqt4))

# Example 4: Edge case with ambiguous salary reference
pqt5 = prompt_question_tables.format(
    tables=text_tables,
    question="List all employees who had a salary increase of more than 10% between 2022 and 2023"
)
print("\nTest Case 4 - Temporal Salary Analysis:")
print(return_OAI(pqt5))

# Example 5: Query with potentially missing information
pqt6 = prompt_question_tables.format(
    tables=text_tables,
    question="Show me the department breakdown of employees with PhD degrees and their total bonuses"
)
print("\nTest Case 5 - Department and Education Query:")
print(return_OAI(pqt6))


Test Case 3 - Complex Query:
{
    "tables": ["employees", "salary", "studies"]
}

Test Case 4 - Temporal Salary Analysis:
{
    "tables": ["employees", "salary"]
}

Test Case 5 - Department and Education Query:
{
    "tables": ["employees", "salary"]
}


## Analysis Report

## Overview of GPT's Table Selection Performance

### Test Cases and Results
1. **Simple Query**: "What is the average salary of all employees?"
   - Tables selected: employees, salary
   - Result: Correct selection, recognizing both current and historical salary data needed

2. **Education Query**: "Which employees have a GPA above 3.5?"
   - Tables selected: employees, studies
   - Result: Accurate selection, linking employee records with educational data

3. **Complex Query**: "What is the average salary of employees who have a master's degree?"
   - Tables selected: employees, salary
   - Result: Incomplete - missed including 'studies' table which is needed for degree information

### Key Findings

#### Strengths
- Excellent at identifying direct table relationships
- Handles simple queries with high accuracy
- Correctly identifies primary data sources

#### Limitations
1. **Implicit Join Recognition**
   - Sometimes misses necessary table joins
   - May not recognize indirect relationships between tables

2. **Context Sensitivity**
   - Performance depends heavily on table definition quality
   - May miss subtle requirements in complex queries

3. **Edge Cases**
   - Struggles with queries requiring temporal analysis
   - May overlook tables needed for complete data context

### Recommendations for Improvement
1. Enhance table definitions with relationship information
2. Include primary/foreign key details in table descriptions
3. Add more context about data relationships in the schema

### Lessons Learned
- GPT-3.5 shows promise in SQL table selection
- Quality of input definitions is crucial
- Complex queries require more sophisticated table relationship understanding